<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/autogen/AutoGen_074_FunctionCalling_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧪 OpenAI Function Calling with AutoGen **0.7.4** — Colab Lab

이 노트북은 **AutoGen 0.7.4 (stable)** 에서 **OpenAI Function Calling**을 사용하는 방법을 실습합니다.  
구성: 설치 → 버전 확인 → API Key 설정 → **FunctionTool** 데모 → **다중툴/반복호출** → **BDI 팀 실습** → 과제

> **주의:** 본 노트북은 교육 목적 예제입니다. 외부 API 요금/정책을 확인하고 사용하세요.  
> GPU 불필요. OpenAI API Key 필요.

## 1) 설치 — AutoGen 0.7.4 고정

In [1]:
%pip -q install --upgrade pip
%pip -q install "autogen-agentchat==0.7.4" "autogen-core==0.7.4" "autogen-ext[openai]==0.7.4"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.6 MB/s eta 0:00:00


## 2) 버전 확인

In [2]:
import pkg_resources, sys, platform
pkgs = ["autogen-agentchat", "autogen-core", "autogen-ext"]
print("Python:", sys.version)
print("Platform:", platform.platform())
for p in pkgs:
    try:
        print(p, pkg_resources.get_distribution(p).version)
    except Exception as e:
        print(p, "not found:", e)

/tmp/ipython-input-3950987313.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources, sys, platform


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Platform: Linux-6.1.123+-x86_64-with-glibc2.35
autogen-agentchat 0.7.4
autogen-core 0.7.4
autogen-ext 0.7.4


## 3) OpenAI API Key 설정

In [6]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

## 4) Function Calling 기본 — 단일 함수 도구

여기서는 **FunctionTool**로 날씨 조회 함수를 등록하고, 모델이 **도구 호출을 스스로 선택**하도록 합니다.  
함수는 학습용으로 **모의 응답**을 반환합니다.

In [14]:
import asyncio
from typing import Dict
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage


def get_weather(location: str, date: str) -> Dict[str, str]:
    '''학습용 모의 날씨 함수: 실제 API 대신 고정 응답'''
    table = {
        "서울": {"forecast": "맑음", "high": "28", "low": "18"},
        "부산": {"forecast": "구름많음", "high": "27", "low": "21"},
    }
    info = table.get(location, {"forecast": "알 수 없음", "high": "-", "low": "-"})
    return {"location": location, "date": date, **info}

# Corrected instantiation of FunctionTool
weather_tool = FunctionTool(
    name="get_weather",
    func=get_weather,
    description="지정한 위치(location)와 날짜(date)에 대한 간단한 날씨 정보를 반환합니다.",
    # Removed the 'sig' parameter as it's not expected
)


model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

assistant = AssistantAgent(
    "WeatherAssistant",
    model_client=model_client,
    tools=[weather_tool],
    system_message=(
        "You are a weather assistant. If helpful, call the tool to fetch weather."
        " 답변은 한국어로."
    ),
)


async def demo_weather():
    user_prompt = "서울의 2025-09-10 날씨 알려줘. 최고/최저 포함."
    # Corrected instantiation of TextMessage and added cancellation_token
    response = await assistant.on_messages([TextMessage(content=user_prompt, role="user", source="user_input")], cancellation_token=None)
    # Access and print the content from the chat_message
    if response and response.chat_message and hasattr(response.chat_message, 'content'):
        print(response.chat_message.content)


await asyncio.wait_for(demo_weather(), timeout=60)

{'location': '서울', 'date': '2025-09-10', 'forecast': '맑음', 'high': '28', 'low': '18'}


## 5) 여러 함수 도구 + 반복 호출(멀티스텝)

AutoGen 0.7.4에서는 에이전트가 필요 시 **연속적으로 도구를 호출**하는 플로우를 구성할 수 있습니다.  
여기서는 `moving_average`, `volatility` 두 도구를 등록하고 모델이 **먼저 통계를 계산 → 결과를 종합**하도록 유도합니다.

In [19]:
import statistics
from typing import List, Dict
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage


def moving_average(prices: List[float], window: int) -> Dict[str, float]:
    '''단순 이동평균'''
    if window < 1 or window > len(prices):
        raise ValueError("window must be between 1 and len(prices)")
    ma = sum(prices[-window:]) / window
    return {"ma": ma, "n": window}


def volatility(prices: List[float]) -> Dict[str, float]:
    '''표준편차 기반 단순 변동성'''
    if len(prices) < 2:
        return {"vol": 0.0, "n": len(prices)}
    return {"vol": float(statistics.pstdev(prices)), "n": len(prices)}


ma_tool = FunctionTool(name="moving_average", func=moving_average, description="주가의 단순 이동평균을 계산합니다.")
vol_tool = FunctionTool(name="volatility", func=volatility, description="가격 리스트의 변동성(표준편차)을 계산합니다.")

model_client = OpenAIChatCompletionClient(model="gpt-4.1-mini")

quant = AssistantAgent(
    "QuantAnalyst",
    model_client=model_client,
    tools=[ma_tool, vol_tool],
    system_message=(
        "You are a quantitative analyst. Decide if you need to call moving_average or volatility, "
        "possibly multiple times, then synthesize a concise Korean recommendation."
    ),
)


async def demo_tools_loop():
    prices = [101, 103, 102, 105, 110, 108, 111, 115]
    user_prompt = (
        f"가격 데이터 {prices} 기반으로 단기(window=3) 이동평균과 변동성을 파악하고, "
        f"리스크를 고려한 투자 의견을 2문장으로 제시해."
    )
    response = await quant.on_messages([TextMessage(content=user_prompt, role="user", source="user_input")], cancellation_token=None)
    # Access and print the content from the chat_message
    if response and response.chat_message and hasattr(response.chat_message, 'content'):
        print(response.chat_message.content)


await asyncio.wait_for(demo_tools_loop(), timeout=60)

{'ma': 111.33333333333333, 'n': 3}
{'vol': 4.621620386834038, 'n': 8}


**Belief → Desire → Intention** 역할을 나눠 팀을 구성합니다.  
- `Belief`: 데이터/사실 요약(필요 시 도구 호출)  
- `Desire`: SMART 목표 수립  
- `Intention`: 실행 계획 표(메트릭 포함)  
여기서는 `moving_average`, `volatility` 같은 정량 도구를 Belief가 활용할 수 있게 등록합니다.

## 7) 문제 해결 (Troubleshooting)

- **ImportError / ModuleNotFoundError**: 런타임 재시작 후 다시 실행하세요. 버전은 **정확히 0.7.4**인지 확인.  
- **인증 오류**: `OPENAI_API_KEY`가 올바른지, 결제/쿼터 상태를 확인.  
- **응답 지연**: 도구 호출이 연쇄적으로 발생하면 시간이 소요됩니다. 셀의 `timeout` 값을 늘리세요.  
- **도구 인자 스키마 불일치**: 함수 시그니처와 입력 JSON 타입을 점검하세요.

## 8) 과제

1) `get_weather`를 실제 외부 API로 교체하고(예: Open-Meteo), 도구 호출 결과를 **BDI 팀**에서 활용하도록 확장하세요.  
2) `RiskOfficer` 에이전트를 추가하여 Intention 결과에 리스크 완화 액션(모니터링 임계값, 헤지 룰)을 병합하세요.  
3) `RedisMemory` 또는 파일 기반 캐시를 붙여, 동일 입력에 대한 **재사용/가속**을 구현하세요.